In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-ml-challenge-2021-hackerearth/sample_submission.csv
/kaggle/input/amazon-ml-challenge-2021-hackerearth/train.csv
/kaggle/input/amazon-ml-challenge-2021-hackerearth/test.csv


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [4]:
df = pd.read_csv('../input/amazon-ml-challenge-2021-hackerearth/train.csv',escapechar="\\",quoting=3)
df.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4


In [5]:
df = df.mask(pd.isnull,df['TITLE'],axis=0)
df = df.mask(pd.isnull,df['DESCRIPTION'],axis=0)
df = df.mask(pd.isnull,df['BULLET_POINTS'],axis=0)
df = df.dropna()

print(np.sum(df.isnull())/len(df))
print(df.head())

TITLE             0.0
DESCRIPTION       0.0
BULLET_POINTS     0.0
BRAND             0.0
BROWSE_NODE_ID    0.0
dtype: float64
                                               TITLE  \
0         Pete The Cat Bedtime Blues Doll, 14.5 Inch   
1  The New Yorker NYHM014 Refrigerator Magnet, 2 ...   
2  The Ultimate Self-Sufficiency Handbook: A Comp...   
3   Amway Nutrilite Kids Chewable Iron Tablets (100)   
4  Teacher Planner Company A4 6 Lesson Academic T...   

                                         DESCRIPTION  \
0  Pete the Cat is the coolest, most popular cat ...   
1  The New Yorker Handsome Cello Wrapped Hard Mag...   
2  The Ultimate Self-Sufficiency Handbook: A Comp...   
3   Amway Nutrilite Kids Chewable Iron Tablets (100)   
4  Teacher Planner Company A4 6 Lesson Academic T...   

                                       BULLET_POINTS  \
0  [Pete the Cat Bedtime Blues plush doll,Based o...   
1  [Cat In A Tea Cup by New Yorker cover artist G...   
2                                

In [ ]:
from nltk.corpus import stopwords
stop_words_set = set(stopwords.words('english'))

count_list = df['BROWSE_NODE_ID'].value_counts().head(3000)
print(count_list)
print("\n ##################################################### \n")
print(f"Number of data points : {np.sum(count_list)}")
df.head()

################## Multinomial Naive bayes ############################
clf = MultinomialNB()
count_vect = CountVectorizer(max_features=2000,stop_words=stop_words_set)

tokens_title         = set()
tokens_decription    = set()
tokens_bullet_points = set()

for j in range(30):
    train_rows = []
    for i in range(len(count_list)):
        num_rows = round(count_list.iloc[i]/50)
        #print("######################################################\n")
        #print(f" Number of elements selected from class {df.index.values[i]} : {num_rows}\n")
        row_list = np.random.choice(df[df['BROWSE_NODE_ID']==count_list.index.values[i]].index.values,num_rows)
        #print(row_list[:5])
        train_rows.extend(row_list)
        print(f"############## {i+1} ################",end='\r')
    print("\n")
    #print(len(train_rows))

    # Shuffle row_list
    np.random.shuffle(train_rows)

    ############## Train data  #######################
    df_train  = df.loc[train_rows]
    
    ############## Epoch i training ##################
    X_train =  df_train[['TITLE','DESCRIPTION','BULLET_POINTS']] 
    #y_train = df_train['BROWSE_NODE_ID'], random_state = 0,test_size=0.0)
    X_train_counts_1 = count_vect.fit_transform(X_train['TITLE']).toarray()
    tokens_title.update(set(count_vect.get_feature_names()))
    
    X_train_counts_2 = count_vect.fit_transform(X_train['DESCRIPTION']).toarray()
    tokens_decription.update(set(count_vect.get_feature_names()))
    
    X_train_counts_3 = count_vect.fit_transform(X_train['BULLET_POINTS']).toarray()
    tokens_bullet_points.update(set(count_vect.get_feature_names()))
    
    print(f"Title: {len(tokens_title)} :: Description: {len(tokens_decription)} :: Bullet Points: {len(tokens_bullet_points)}")
    
    print(f"############################  {j} ###################################")
    
    #X_train_counts = np.hstack((X_train_counts_1,X_train_counts_2,X_train_counts_3))
    #print(X_train_counts.shape)
    
    #tfidf_transformer = TfidfTransformer()
    #X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts).toarray()
    #print(X_train_tfidf.shape)

    #clf.fit(X_train_tfidf, y_train)
    
    # Prediction
    #X_test_counts_1 = count_vect.transform(X_test['TITLE']).toarray()
    #X_test_counts_2 = count_vect.transform(X_test['DESCRIPTION']).toarray()
    #X_test_counts_3 = count_vect.transform(X_test['BULLET_POINTS']).toarray()
    #X_test_counts = np.hstack((X_test_counts_1,X_test_counts_2,X_test_counts_3))
    #print(X_test_counts.shape)
    
    #y_pred = clf.predict(X_test_counts)
    #accuracy = np.sum(y_pred==y_test)/len(y_test)
    #print(f"###########Epoch {j} : {accuracy} ##########")

1045      215698
5          70318
1251       51929
1052       45553
4          34169
           ...  
110169        73
3902          73
25122         72
34978         72
16263         72
Name: BROWSE_NODE_ID, Length: 3000, dtype: int64

 ##################################################### 

Number of data points : 2778244
############## 3000 ################

Title: 2000 :: Description: 2000 :: Bullet Points: 2000
############################  0 ###################################
############## 3000 ################

Title: 2138 :: Description: 2081 :: Bullet Points: 2075
############################  1 ###################################
############## 3000 ################

Title: 2218 :: Description: 2116 :: Bullet Points: 2112
############################  2 ###################################
############## 3000 ################

Title: 2265 :: Description: 2147 :: Bullet Points: 2134
############################  3 ###################################
############## 3000 ######

In [ ]:
print(len(tokens_title))
print(len(tokens_decription))
print(len(tokens_bullet_points))

In [ ]:
count_list = df['BROWSE_NODE_ID'].value_counts().count(3000)
print(count_list)
print("\n ##################################################### \n")
print(f"Number of data points : {len(count_list)}")
df.head()
classes_list = list(count_list.index)

################## Multinomial Naive bayes ############################
clf = MultinomialNB()
print(len(tokens_title))
print(len(tokens_decription))
print(len(tokens_bullet_points))
count_vect_title         = CountVectorizer(vocabulary=tokens_title)
count_vect_decription    = CountVectorizer(vocabulary=tokens_decription)
count_vect_bullet_points = CountVectorizer(vocabulary=tokens_bullet_points)

for j in range(50):
    train_rows = []
    for i in range(len(count_list)):
        num_rows = round(count_list.iloc[i]/100)
        if(num_rows<1):
            num_rows = 1
        #print("######################################################\n")
        #print(f" Number of elements selected from class {df.index.values[i]} : {num_rows}\n")
        row_list = np.random.choice(df[df['BROWSE_NODE_ID']==count_list.index.values[i]].index.values,num_rows)
        #print(row_list[:5])
        train_rows.extend(row_list)
        print(f"############## {i+1} ################",end='\r')
    print("\n")
    #print(len(train_rows))

    # Shuffle row_list
    np.random.shuffle(train_rows)

    ############## Train data  #######################
    df_train  = df.loc[train_rows]
    
    ############## Epoch i training ##################
    X_train, X_test, y_train, y_test = train_test_split(df_train[['TITLE','DESCRIPTION','BULLET_POINTS']], df_train['BROWSE_NODE_ID'], random_state = 0)
    X_train_counts_1 = count_vect_title.fit_transform(X_train['TITLE']).toarray()
    X_train_counts_2 = count_vect_decription.fit_transform(X_train['DESCRIPTION']).toarray()
    X_train_counts_3 = count_vect_bullet_points.fit_transform(X_train['BULLET_POINTS']).toarray()
    X_train_counts = np.hstack((X_train_counts_1,X_train_counts_2,X_train_counts_3))
    print([X_train_counts.shape,X_train_counts_1.shape,X_train_counts_2.shape,X_train_counts_3.shape])
    
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts).toarray()
    print(X_train_tfidf.shape)

    clf.partial_fit(X_train_tfidf, y_train,classes=classes_list)
    
    # Prediction
    X_test_counts_1 = count_vect_title.transform(X_test['TITLE']).toarray()
    X_test_counts_2 = count_vect_decription.transform(X_test['DESCRIPTION']).toarray()
    X_test_counts_3 = count_vect_bullet_points.transform(X_test['BULLET_POINTS']).toarray()
    X_test_counts = np.hstack((X_test_counts_1,X_test_counts_2,X_test_counts_3))
    print(X_test_counts.shape)
    
    y_pred = clf.predict(X_test_counts)
    accuracy = np.sum(y_pred==y_test)/len(y_test)
    print(f"###########Epoch {j} : {accuracy} ##########")

In [21]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(n_jobs=-1)

count_list = df['BROWSE_NODE_ID'].value_counts().head(1000)
print(count_list)
print("\n ##################################################### \n")
print(f"Number of data points : {len(df)}")
df.head()

################## Multinomial Naive bayes ############################
count_vect_title         = CountVectorizer(vocabulary=tokens_title)
count_vect_decription    = CountVectorizer(vocabulary=tokens_decription)
count_vect_bullet_points = CountVectorizer(vocabulary=tokens_bullet_points)

for j in range(50):
    train_rows = []
    for i in range(len(count_list)):
        num_rows = round(count_list.iloc[i]/100)
        #print("######################################################\n")
        #print(f" Number of elements selected from class {df.index.values[i]} : {num_rows}\n")
        row_list = np.random.choice(df[df['BROWSE_NODE_ID']==count_list.index.values[i]].index.values,num_rows)
        #print(row_list[:5])
        train_rows.extend(row_list)
        print(f"############## {i+1} ################",end='\r')
    print("\n")
    #print(len(train_rows))

    # Shuffle row_list
    np.random.shuffle(train_rows)

    ############## Train data  #######################
    df_train  = df.loc[train_rows]
    
    ############## Epoch i training ##################
    X_train, X_test, y_train, y_test = train_test_split(df_train[['TITLE','DESCRIPTION','BULLET_POINTS']],
                                                        df_train['BROWSE_NODE_ID'], random_state = 0,test_size=0.2)
    X_train_counts_1 = count_vect_title.fit_transform(X_train['TITLE']).toarray()
    X_train_counts_2 = count_vect_decription.fit_transform(X_train['DESCRIPTION']).toarray()
    X_train_counts_3 = count_vect_bullet_points.fit_transform(X_train['BULLET_POINTS']).toarray()
    X_train_counts = np.hstack((X_train_counts_1,X_train_counts_2,X_train_counts_3))
    print(X_train_counts.shape)
    
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts).toarray()
    print(X_train_tfidf.shape)

    sgd_clf.partial_fit(X_train_tfidf, y_train,classes=classes_list)
    
    # Prediction
    X_test_counts_1 = count_vect_title.transform(X_test['TITLE']).toarray()
    X_test_counts_2 = count_vect_decription.transform(X_test['DESCRIPTION']).toarray()
    X_test_counts_3 = count_vect_bullet_points.transform(X_test['BULLET_POINTS']).toarray()
    X_test_counts = np.hstack((X_test_counts_1,X_test_counts_2,X_test_counts_3))
    print(X_test_counts.shape)
    
    y_pred = sgd_clf.predict(X_test_counts)
    accuracy = np.sum(y_pred==y_test)/len(y_test)
    print(f"###########Epoch {j} : {accuracy} ##########")

1045     215698
5         70318
1251      51929
1052      45553
4         34169
          ...  
1344        417
8939        416
615         415
10495       414
6980        414
Name: BROWSE_NODE_ID, Length: 1000, dtype: int64

 ##################################################### 

Number of data points : 2903014
############## 1000 ################

(19349, 3470)
(19349, 3470)
(4838, 3470)
###########Epoch 0 : 0.5183960314179413 ##########
############## 1000 ################

(19349, 3470)
(19349, 3470)
(4838, 3470)
###########Epoch 1 : 0.5324514262091773 ##########
############## 1000 ################

(19349, 3470)
(19349, 3470)
(4838, 3470)
###########Epoch 2 : 0.5363786688714345 ##########
############## 1000 ################

(19349, 3470)
(19349, 3470)
(4838, 3470)
###########Epoch 3 : 0.5367920628358825 ##########
############## 1000 ################

(19349, 3470)
(19349, 3470)
(4838, 3470)
###########Epoch 4 : 0.5239768499379909 ##########
############## 1000 ###############

KeyboardInterrupt: 

In [14]:
#### Test and evaluation

test_df = pd.read_csv('../input/amazon-ml-challenge-2021-hackerearth/test.csv',escapechar="\\",quoting=3)
test_df = test_df.mask(pd.isnull,test_df['TITLE'],axis=0)
test_df = test_df.mask(pd.isnull,test_df['DESCRIPTION'],axis=0)
test_df = test_df.mask(pd.isnull,test_df['BULLET_POINTS'],axis=0)
test_df = test_df.dropna()
test_df.head()

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero


In [15]:
test_df = test_df.dropna()

count_vect_title         = CountVectorizer(vocabulary=tokens_title)
count_vect_decription    = CountVectorizer(vocabulary=tokens_decription)
count_vect_bullet_points = CountVectorizer(vocabulary=tokens_bullet_points)

X_test_1 = test_df['TITLE']
X_test_2 = test_df['DESCRIPTION']
X_test_3 = test_df['BULLET_POINTS']

X_test_counts_1 = count_vect_title.fit_transform(X_test_1).toarray()
X_test_counts_2 = count_vect_decription.fit_transform(X_test_2).toarray()
X_test_counts_3 = count_vect_bullet_points.fit_transform(X_test_3).toarray()
X_test_counts   = np.hstack((X_test_counts_1,X_test_counts_2,X_test_counts_3))
print(X_test_counts.shape)
y_pred = clf.predict(X_test_counts)

(110775, 3470)


In [16]:
test_df = test_df.drop(['TITLE','DESCRIPTION','BULLET_POINTS','BRAND'],axis=1)
test_df['BROWSE_NODE_ID'] = y_pred
test_df = test_df.set_index('PRODUCT_ID')
test_df.head()

,BROWSE_NODE_ID
PRODUCT_ID,
1,74
2,2321
3,81
4,604
5,1239


In [17]:
test_df.to_csv('./result_nb.csv')

In [ ]:
result_1 = clf